In [66]:
import numpy as np
import pandas as pd
from collections import Counter

In [170]:
def item_features(Nitems,g):
    return np.random.geometric(g,(20,Nitems))


def episodic_matrix(wT,g,u,c):
    M = wT*0
    for i in range(len(wT[0])):
        copy = np.random.binomial(1,u,20)
        trace = copy*wT[:,i]
        correct = np.random.binomial(1,c,20)
        wrong_copied = np.where((trace>0) * (correct==0))
        trace[wrong_copied] = np.random.geometric(g,len(wrong_copied))
        M[:,i] = trace
        
    return M


def qtype(probe,trace):
    nonzero_probe = probe[trace > 0]
    nonzero_trace = trace[trace > 0]
    
    return sum(~(nonzero_probe == nonzero_trace))

def mtype(probe,trace):
    m = np.tile(0,200)
    nonzero_probe = probe[trace > 0]
    nonzero_trace = trace[trace > 0]
    nonzero_match = nonzero_probe[nonzero_probe==nonzero_trace]
    nonzero_match_count = pd.value_counts(nonzero_match)
    m[np.array(nonzero_match_count.keys().tolist(),dtype=np.int32)-1] = nonzero_match_count.tolist()
    
    return m


def similarity(probe,trace,c,g):
    Qtype = qtype(probe,trace)
    Mtype = mtype(probe,trace)
    x1 = (1-c)**Qtype
    x2 = 1
    for i in range(len(Mtype)):
        x2 = x2 * ((c+(1-c)*g*(1-g)**(i))/(0.0001+g*(1-g)**(i)))**Mtype[i]
    
    return(x1*x2)


def memory_strength(probe,M,c,g):
    phi = np.zeros(len(M[0]))
    for i in range(len(M[0])):
        phi[i] = similarity(probe,M[:,i],c,g)
    
    return np.mean(phi)


def response_vector(g,u,c,Nitems):
    W = item_features(Nitems,g)
    wT = W[:,0:int(Nitems/2)]
    wD = W[:,int(Nitems/2):Nitems]
    M = episodic_matrix(wT,g,u,c)
    target = np.array([memory_strength(wT[:,i],M,c,g) for i in range(int(Nitems/2))])
    distractor = np.array([memory_strength(wD[:,i],M,c,g) for i in range(int(Nitems/2))])
    target = target > 1
    distractor = distractor > 1
    target = target.astype(int)
    distractor = distractor.astype(int)
    
    return np.c_[target,distractor]


def REM(pars,Nitems,Ntrials,flag):
    # pars:[g,u,c]
    # g: Environmental Base Rate
    # u: Feature Copying
    # c: Correct Feature Copying
    # Ntimes: targets = Nitems/2; distractors = Nitems/2
    # flag: 1, full matrix output; 2, contingency table
    g,u,c = pars
    out1 = np.zeros((Ntrials,int(Nitems/2),2))
    out2 = np.zeros((Ntrials,2,2))
    for n in range(Ntrials):
        o = response_vector(g,u,c,Nitems)
        out1[n,:,:] = o
        out2[n,:,:] = np.array([[sum(o[:,0]),(Nitems/2)-sum(o[:,0])],
                                [sum(o[:,1]),(Nitems/2)-sum(o[:,1])]])
    if flag:
        # first column: vector of Hit (1) and Miss (0) for targets
        # second column: vector of Correct Rejection (0) and False Alarms (1) for distractors
        return out1
    else:
        # first row (target): Hit and Miss
        # second row (distractor): False Alarms and Correct Rejection
        return out2
                            
    

In [187]:
REM([0.4,0.4,0.6],40,1,0)

array([[[14.,  6.],
        [ 4., 16.]]])